In [3]:
import csv
import json
import pandas as pd
import numpy as np
from pprint import pprint
from collections import defaultdict, Counter
from dateutil.parser import *

SMA_WINDOW = 7

def get_lineitem(d):
    cases = None
    deaths = None
    caserate = None
    mortality = None
    caserate_ma = None
    mortality_ma = None
    tests = None
    totalHospitalized = None
    testingRate = None
    hospitalizationRate = None
        
    try:
        cases = int(d["cases"])
        deaths = int(d["deaths"])
        caserate = float(d["caserate"])
        mortality = float(d["covidmortality"])
        caserate_ma = float(d["caserate7day"])
        mortality_ma = float(d["covidmortality7day"])
        tests = int(d["tests"])
        totalHospitalized = int(d["hospTot"])
        testingRate = float(d["testingRate"])
        hospitalizationRate = float(d["hospRate"])
        
    except ValueError:
        cases = 0
        deaths = 0
        caserate = 0
        mortality = 0
        caserate_ma = 0
        mortality_ma = 0
        tests = 0
        totalHospitalized = 0
        testingRate = 0
        hospitalizationRate = 0

    if caserate_ma < 0:
        caserate_ma = 0
    if mortality_ma < 0:
        mortality_ma = 0
    if tests < 0:
        tests = 0
    if totalHospitalized < 0:
        totalHospitalized = 0
    if testingRate < 0:
        testingRate = 0
    if hospitalizationRate < 0:
        hospitalizationRate = 0

    return {"t": int(parse(d["date"]).timestamp()),
            "cases": cases,
            "deaths": deaths,
            "caseRate": caserate,
            "mortality": mortality,
            "caseRateMA": caserate_ma,
            "mortalityMA": mortality_ma,
            "tests": tests,
            "totalHospitalized": totalHospitalized,
            "testingRate": testingRate,
            "hospitalizationRate": hospitalizationRate}

def linechart(fn="covidtimeseries.csv"):

    data = defaultdict(list)
    with open(fn, "r") as fp:
        reader = csv.reader(fp)
        header = next(reader)
        for row in reader:
            d = {k:v.strip() for k, v in zip(header, row)}
            
            if d["date"] == "":
                continue

            item = get_lineitem(d)
            k = ""
            if d["nation"] != "NA":
                k = "_nation"
            elif d["statename"] != "":
                k = d["fips"].zfill(2)
            else:
                k = d["fips"].zfill(5)
            data[k].append(item)
            
        

    item0 = {"t": 1579582800, "deaths": 1, "cases": 1,
                "caseRate": 0, "mortality": 0,
                "caseRateMA": 0, "mortalityMA": 0,
                "tests": 0, "totalHospitalized": 0,
                "testingRate": 0, "hospitalizationRate": 0}
    item1 = {"t": 1588731173, "deaths": 1, "cases": 1,
                "caseRate": 0, "mortality": 0,
                "caseRateMA": 0, "mortalityMA": 0,
                "tests": 0, "totalHospitalized": 0,
                "testingRate": 0, "hospitalizationRate": 0}
    
    itemNA = {"t": 1585713600,
            "cases": 0,
            "deaths": 0,
            "caseRate": 0,
            "mortality": 0,
            "caseRateMA": 0,
            "mortalityMA": 0,
            "tests": 0,
            "totalHospitalized": 0,
            "testingRate": 0,
            "hospitalizationRate": 0}
    data["99999"] = [itemNA]

    data["_"] = [item0, item1]

    output = defaultdict(dict)
    for k, v in data.items():
        if k == "_nation":
            continue
        output[k[:2]][k] = v

    for k, v in output.items():
        v["_nation"] = data["_nation"] 
        v["_"] = data["_"]
        v["99999"] = data["99999"]
        with open(f"../timeseries{k}.json", "w") as fp:
            json.dump(v, fp, indent=2)


if __name__=="__main__":

    linechart()

KeyError: 'tests'